In [24]:
#Import required libraries
import pandas as pd
#pandas-> used to load and manipulate tabular data(CSV,DataFrame)

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler,LabelEncoder

import pickle

#Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [26]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [27]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [28]:
geo_encoded_df = pd.DataFrame(
    geo_encoder,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

In [29]:
data = pd.concat(
    [data.drop('Geography',axis = 1),geo_encoded_df],axis = 1
)

In [30]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [31]:
X = data.drop('Exited',axis=1) #Independent Variables
y = data['Exited'] #Dependent Variable

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [33]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Implementation

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [36]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [39]:
model.compile(
    optimizer = opt,
    loss = "binary_crossentropy",
    metrics = ['accuracy']
)

In [40]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [41]:
tensorboard_callback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=1
    )

In [42]:
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [43]:
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test),
    epochs=100, 
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 5ms/step - loss: 0.3951 - accuracy: 0.8345 - val_loss: 0.3613 - val_accuracy: 0.8525
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3601 - accuracy: 0.8543 - val_loss: 0.3469 - val_accuracy: 0.8630
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3464 - accuracy: 0.8575 - val_loss: 0.3537 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3455 - accuracy: 0.8583 - val_loss: 0.3456 - val_accuracy: 0.8595
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3389 - accuracy: 0.8595 - val_loss: 0.3465 - val_accuracy: 0.8635
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3369 - accuracy: 0.8604 - val_loss: 0.3506 - val_accuracy: 0.8535
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3374 - accuracy: 0.8626 - val_loss: 0.3447 - val_accuracy: 0.8600

In [44]:
model.save('model.h5')

c:\Users\aagma\anaconda3\envs\venva\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [46]:
%tensorboard --logdir logs
!dir logs 


Reusing TensorBoard on port 6007 (pid 18204), started 10:11:11 ago. (Use '!kill 18204' to kill it.)

 Volume in drive C is OS
 Volume Serial Number is 60DF-1738

 Directory of c:\Users\aagma\OneDrive\Desktop\ANN\logs

12-01-2026  12:55    <DIR>          .
12-01-2026  12:55    <DIR>          ..
13-01-2026  10:34    <DIR>          fit
               0 File(s)              0 bytes
               3 Dir(s)  279,674,429,440 bytes free
